In [1]:
import optuna
from lightgbm import LGBMClassifier
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score

In [5]:
df_final = pd.read_csv('nice_jp_mean.csv')
df_final

,PBCO_GB,HDOF_BR_GB,PB_ORG_TYP,ESTB_GB,EMP_CNT,유동자산,매출채권,비유동자산,유형자산,자산총계,...,BZ_TYP_O,BZ_TYP_S,BZ_TYP_V,FR_IVST_CORP_YN_N,FR_IVST_CORP_YN_Y,VENT_YN_N,VENT_YN_Y,MDSCO_PRTC_YN_N,MDSCO_PRTC_YN_Y,isCLSBZ
0,0.014,-0.373690,-0.031043,-0.085399,-0.086621,-0.259653,-0.119202,-0.136335,-0.195278,-0.180919,...,-0.0928,-0.014967,-0.033482,0.228939,-0.228939,0.305212,-0.305212,0.055324,-0.055324,0
1,0.014,-0.373690,-0.031043,-0.085399,-0.138795,-0.204746,-0.131608,-0.200961,-0.230504,-0.188864,...,-0.0928,-0.014967,-0.033482,0.228939,-0.228939,0.305212,-0.305212,0.055324,-0.055324,0
2,0.014,-0.373690,-0.031043,-0.085399,0.604697,-0.282210,-0.136413,-0.261092,-0.236755,-0.252230,...,-0.0928,-0.014967,-0.033482,0.228939,-0.228939,0.305212,-0.305212,0.055324,-0.055324,0
3,0.014,-0.373690,-0.031043,-0.085399,0.122079,-0.128061,0.036686,-0.048901,0.019349,-0.080529,...,-0.0928,-0.014967,-0.033482,0.228939,-0.228939,0.305212,-0.305212,0.055324,-0.055324,0
4,0.014,-0.373690,-0.031043,-0.085399,-0.347495,-0.075416,-0.172271,-0.317192,-0.317380,-0.189841,...,-0.0928,-0.014967,-0.033482,0.228939,-0.228939,0.305212,-0.305212,0.055324,-0.055324,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35716,0.014,2.676017,-0.031043,-0.085399,0.000000,-0.298135,-0.182363,-0.326661,-0.336873,-0.291480,...,-0.0928,-0.014967,-0.033482,0.228939,-0.228939,0.305212,-0.305212,0.055324,-0.055324,1
35717,0.014,2.676017,-0.031043,-0.085399,-0.223580,-0.270115,-0.163043,-0.292007,-0.281672,-0.262201,...,-0.0928,-0.014967,-0.033482,0.228939,-0.228939,0.305212,-0.305212,0.055324,-0.055324,1
35718,0.014,2.676017,-0.031043,-0.085399,-0.367061,-0.311679,-0.186115,-0.328380,-0.334445,-0.298225,...,-0.0928,-0.014967,-0.033482,0.228939,-0.228939,0.305212,-0.305212,0.055324,-0.055324,1
35719,0.014,-0.373690,-0.031043,-0.085399,0.000000,-0.314269,-0.187721,-0.332866,-0.342943,-0.301564,...,-0.0928,-0.014967,-0.033482,0.228939,-0.228939,0.305212,-0.305212,0.055324,-0.055324,1


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_final.iloc[:,:-1],df_final.iloc[:,-1] , test_size=0.2, random_state=0, stratify=df_final.isCLSBZ)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((28576, 78), (7145, 78), (28576,), (7145,))

In [7]:
def obj_lgb(trial):
    dtrain = lgb.Dataset(X_train, label=y_train)
    params = {
        # 'device' : "DUSE_GPU=1",
        "objective": "binary",
        "metric": "acc",
        "n_jobs": -1,
        "random_state" : 2022,
        "n_estimators" : trial.suggest_categorical("n_estimators", [1000, 2000, 3000, 4000, 5000]),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 20),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-1),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.8, 1.0),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.8, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "feature_fraction_seed": trial.suggest_int("feature_fraction_seed", 1, 7),
        "bagging_seed": trial.suggest_int("bagging_seed", 1, 7),
        "drop_seed": trial.suggest_int("drop_seed", 1, 7),
        "data_random_seed": trial.suggest_int("data_random_seed", 1, 7),
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        # "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 15, 50),
        'cv' : 5
    }
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(X_train, y_train, verbose=-1, eval_set=[(X_test, y_test)], eval_metric="acc")

    preds = lgb_model.predict(X_test)
    score = accuracy_score(y_test, preds)
    return score



In [8]:
study = optuna.create_study(direction="maximize")
study.optimize(obj_lgb, n_trials=500, n_jobs=-1)
print('Best trial: score {} \n nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2022-11-24 00:35:05,860] A new study created in memory with name: no-name-fd1efa84-9271-4d2b-a9cb-4508fe07c3a7
[I 2022-11-24 00:37:39,830] Trial 2 finished with value: 0.9641707487753673 and parameters: {'n_estimators': 5000, 'num_leaves': 13, 'min_child_samples': 7, 'learning_rate': 0.003329545649500616, 'bagging_fraction': 0.9602384218222441, 'feature_fraction': 0.8065957717878669, 'bagging_freq': 7, 'feature_fraction_seed': 1, 'bagging_seed': 7, 'drop_seed': 6, 'data_random_seed': 4, 'boosting_type': 'gbdt'}. Best is trial 2 with value: 0.9641707487753673.
[I 2022-11-24 00:39:50,282] Trial 8 finished with value: 0.9566130160951715 and parameters: {'n_estimators': 4000, 'num_leaves': 10, 'min_child_samples': 4, 'learning_rate': 0.0014777164874108552, 'bagging_fraction': 0.8828412393946207, 'feature_fraction': 0.939614507390613, 'bagging_freq': 2, 'feature_fraction_seed': 7, 'bagging_seed': 7, 'drop_seed': 2, 'data_random_seed': 3, 'boosting_type': 'gbdt'}. Best is trial 2 with val